In [1]:
%cd ../../../../

/home/users/dmoreno2016


In [1]:
import tensorflow as tf

2023-09-23 10:25:53.860279: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-23 10:25:55.249474: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
def split_heads(x, num_heads, depth, name='qkv'):
    """Split the last dimension into (num_heads, depth).
    Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
    """
    print(x.shape)
    x = tf.reshape(x, (-1, num_heads, depth))
    print(x.shape)
    return tf.transpose(x, perm=[1, 0, 2], name=name)

d_model = 8
num_head = 4
largo = 5

depth = d_model // num_head


w_q = tf.constant([[1,4,6,3,2,6,2,1],
                   [2,5,1,2,6,8,6,9],
                   [3,3,2,1,3,6,3,8],
                   [2,4,1,6,8,2,6,9],
                   [3,5,8,3,8,0,4,3],                   
                   [5,7,8,2,3,5,4,3],
                   [4,3,1,4,3,0,5,2],
                   [5,5,5,5,4,9,3,4]]) 

w_k = tf.constant([[7,5,4,2,1,0,9,2],
                   [5,7,8,2,3,5,4,3],
                   [4,3,1,4,3,0,5,2],
                   [5,5,5,5,4,9,3,4],
                   [4,3,6,8,5,6,5,6],                   
                   [3,3,2,1,3,6,3,8],
                   [2,4,1,6,8,2,6,9],
                   [3,5,8,3,8,0,4,3]]) 

e_x = tf.constant([[7,2,3,2,1,5,7,3],
                   [3,5,4,2,1,5,7,2],
                   [2,1,4,6,2,7,9,3],
                   [2,4,1,5,7,4,6,8],
                   [1,4,7,8,5,3,6,9]]) # 2 x 3


bq = tf.constant([3,2,5,2,5,7,8,5])
bk = tf.constant([2,1,7,2,4,3,1,2])

print('w_q: {}'.format(w_q.shape))
print('w_k: {}'.format(w_k.shape))
print('e_x: {}'.format(e_x.shape))
print('bq: {}'.format(bq.shape))
print('bk: {}'.format(bk.shape))

w_q: (8, 8)
w_k: (8, 8)
e_x: (5, 8)
bq: (8,)
bk: (8,)


In [74]:
q = tf.matmul(e_x, w_q) + bq
k = tf.matmul(e_x, w_k) + bk

q = split_heads(q, num_head, depth)
k = split_heads(k, num_head, depth)

result_1 = tf.matmul(q, k, transpose_b=True)
result_1

(5, 8)
(5, 4, 2)
(5, 8)
(5, 4, 2)


<tf.Tensor: shape=(4, 5, 5), dtype=int32, numpy=
array([[[29540, 28231, 30408, 35630, 41174],
        [29150, 27844, 29994, 35138, 40610],
        [35135, 33507, 36105, 42271, 48871],
        [38995, 37249, 40125, 47007, 54327],
        [42940, 40976, 44148, 51700, 59764]],

       [[24485, 24514, 28452, 40485, 44136],
        [20761, 20776, 24268, 34223, 37380],
        [27034, 27041, 31788, 44427, 48618],
        [32822, 32844, 38392, 54086, 59088],
        [33620, 33614, 39752, 55090, 60396]],

       [[26267, 28352, 37890, 42694, 46633],
        [27230, 29270, 39036, 44056, 48184],
        [30359, 32234, 42722, 48450, 53199],
        [38875, 41260, 54674, 62014, 68101],
        [45465, 48450, 64334, 72854, 79901]],

       [[38405, 35576, 42518, 43182, 46345],
        [44233, 41174, 49471, 50066, 53574],
        [50538, 46950, 56289, 57048, 61119],
        [54531, 50690, 60813, 61606, 65978],
        [69332, 64916, 78494, 79104, 84346]]], dtype=int32)>

In [39]:
tf.expand_dims(bk, axis=1)

<tf.Tensor: shape=(3, 1), dtype=int32, numpy=
array([[2],
       [1],
       [7]], dtype=int32)>

In [40]:
q

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[20, 47, 50],
       [25, 49, 31]], dtype=int32)>

In [36]:
q = tf.matmul(e_x, w_q)
k = tf.matmul(e_x, w_k)

qk = tf.matmul(q, k, transpose_b=True)

cte_1 = tf.matmul(q, tf.expand_dims(bk, axis=1))
cte_2 = tf.matmul(k, tf.expand_dims(bq, axis=1))

bb = tf.matmul(tf.expand_dims(bk, axis=0), tf.expand_dims(bq, axis=1))

qk + cte_1 + cte_2 + bb

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[5299, 6353],
       [4922, 5902]], dtype=int32)>

In [45]:
tf.expand_dims(bq, axis=0)

<tf.Tensor: shape=(1, 3), dtype=int32, numpy=array([[3, 2, 5]], dtype=int32)>

In [44]:
k

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[31, 49, 31],
       [38, 61, 38]], dtype=int32)>

In [58]:
bq.shape

TensorShape([4, 1, 2])

In [60]:
bk.shape

TensorShape([4, 1, 2])

In [77]:
q = tf.matmul(e_x, w_q)
k = tf.matmul(e_x, w_k)
q = split_heads(q, num_head, depth)
k = split_heads(k, num_head, depth)

qk = tf.matmul(q, k, transpose_b=True)

bias_q = split_heads(tf.expand_dims(bq, axis=1), num_head, depth)
bias_k = split_heads(tf.expand_dims(bk, axis=1), num_head, depth)

cte_1 = tf.matmul(q, bias_k, transpose_b=True)
cte_2 = tf.matmul(bias_q, k, transpose_b=True)
bb = tf.matmul(bias_k, bias_q, transpose_b=True)

result_2_corrected = qk + cte_1 + cte_2 + bb
result_2_corrected

(5, 8)
(5, 4, 2)
(5, 8)
(5, 4, 2)
(1, 8)
(1, 4, 2)
(8, 1)
(1, 4, 2)


<tf.Tensor: shape=(4, 5, 5), dtype=int32, numpy=
array([[[29540, 28231, 30408, 35630, 41174],
        [29150, 27844, 29994, 35138, 40610],
        [35135, 33507, 36105, 42271, 48871],
        [38995, 37249, 40125, 47007, 54327],
        [42940, 40976, 44148, 51700, 59764]],

       [[24485, 24514, 28452, 40485, 44136],
        [20761, 20776, 24268, 34223, 37380],
        [27034, 27041, 31788, 44427, 48618],
        [32822, 32844, 38392, 54086, 59088],
        [33620, 33614, 39752, 55090, 60396]],

       [[26267, 28352, 37890, 42694, 46633],
        [27230, 29270, 39036, 44056, 48184],
        [30359, 32234, 42722, 48450, 53199],
        [38875, 41260, 54674, 62014, 68101],
        [45465, 48450, 64334, 72854, 79901]],

       [[38405, 35576, 42518, 43182, 46345],
        [44233, 41174, 49471, 50066, 53574],
        [50538, 46950, 56289, 57048, 61119],
        [54531, 50690, 60813, 61606, 65978],
        [69332, 64916, 78494, 79104, 84346]]], dtype=int32)>

In [47]:
### FORMA CORRECTA DE HACERLO SIN LA OTRA DIMENSION DEL SPLIT HEAD

q = tf.matmul(e_x, w_q)
k = tf.matmul(e_x, w_k)
qk = tf.matmul(q, k, transpose_b=True)
cte_1 = tf.matmul(q, tf.expand_dims(bk, axis=1))
cte_2 = tf.matmul(tf.expand_dims(bq, axis=0), k, transpose_b=True)
bb = tf.matmul(tf.expand_dims(bk, axis=0), tf.expand_dims(bq, axis=1))
result_2_corrected = qk + cte_1 + cte_2 + bb
result_2_corrected

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[5299, 6433],
       [4842, 5902]], dtype=int32)>

In [13]:
tf.matmul((e_x + pe_t), w_q) + bq

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[ 54, 104,  90],
       [ 46,  96,  79]], dtype=int32)>

In [15]:
a = tf.matmul(e_x, w_q)
b = tf.matmul(pe_t, w_q)

print(a.shape)
print(b.shape)

(2, 3)
(2, 3)


In [5]:
w + b

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[3, 6, 8],
       [4, 7, 3]], dtype=int32)>